In [2]:
%reset -f

import os
import numpy as np
import datetime as dt

import plotly.graph_objects as go
from plotly.graph_objects import Figure

from plotly.graph_objects import Scatter
from plotly.graph_objects import Ohlc


from sxo.util.binning import bin_values

from sxo.interface.entities.instruments import InstrumentUtil
from sxo.apps.simple.persisted_quote import RedisQuote


In [43]:
os.environ["REDIS_PASS"] = "boss"

In [64]:
instr = InstrumentUtil.parse('FxSpot::GBPEUR')
tick_db = RedisQuote(instr)
df = tick_db.tail(np.timedelta64(3, 'm'))

df

,t,bid,ask,bsz,asz
0,2023-10-16 18:17:07.580,1.15706,1.15743,864140.0,864100.0
1,2023-10-16 18:17:09.164,1.15706,1.15743,864140.0,864100.0
2,2023-10-16 18:17:10.330,1.15704,1.15741,864150.0,864120.0
3,2023-10-16 18:17:11.531,1.15705,1.15742,864150.0,864110.0
4,2023-10-16 18:17:12.162,1.15704,1.15741,864150.0,864120.0
...,...,...,...,...,...
132,2023-10-16 18:20:00.395,1.15707,1.15744,864140.0,864090.0
133,2023-10-16 18:20:01.244,1.15707,1.15744,864140.0,950499.0
134,2023-10-16 18:20:02.764,1.15706,1.15743,864140.0,950510.0
135,2023-10-16 18:20:03.888,1.15706,1.15743,864140.0,950510.0


In [70]:
from typing import Tuple
import pandas as pd

from math import log10, floor
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)


def __strat(df:pd.DataFrame,
            alpha:float,
            beta:float,
            precision:int = -1) -> Tuple[np.float64, np.float64, np.float64, np.float64]:
    wmid = (df['bid'].values * df['asz'].values + df['ask'].values * df['bsz'].values) / (df['asz'].values + df['bsz'].values)
    range = np.max(wmid) - min(wmid)
    last_bid, last_ask, last_mid, = df.bid.values[-1], df.ask.values[-1], wmid[-1]

    price = last_mid
    longEntry = price + alpha * range
    longExit = longEntry - beta * range
    shortEntry = price - alpha * range
    shortExit = shortEntry + beta * range

    rs = round_sig
    if precision > 0:
        price = round_sig(price, precision)
        range = round_sig(range, precision)
        longEntry = round_sig(longEntry, precision)
        longExit = round_sig(longExit, precision)
        shortEntry = round_sig(shortEntry, precision)
        shortExit = round_sig(shortExit, precision)

    print(f"LONG:  {price} / {range}. in: {longEntry} -> {longExit}. Delta: {rs(longEntry - price)} -> {rs(longEntry - longExit)}")
    print(f"SHORT: {price} / {range}. in: {shortEntry} -> {shortExit}. Delta: {rs(price - shortEntry)} -> {rs(shortEntry- shortExit)}")
    
    return (longEntry, longExit, shortEntry, shortExit)


df = tick_db.tail(np.timedelta64(3, 'm'))
res = __strat(df, 1.4, 0.5, 7)

LONG:  1.157144 / 0.0002033343. in: 1.157428 -> 1.157327. Delta: 0.00028 -> 0.0001
SHORT: 1.157144 / 0.0002033343. in: 1.156859 -> 1.156961. Delta: 0.00028 -> -0.0001


In [69]:
df

,t,bid,ask,bsz,asz
0,2023-10-16 18:21:22.455,1.15708,1.15745,864130.0,864090.0
1,2023-10-16 18:21:24.551,1.15708,1.15745,864130.0,864090.0
2,2023-10-16 18:21:25.808,1.15708,1.15745,864130.0,864090.0
3,2023-10-16 18:21:27.125,1.15708,1.15745,864130.0,864090.0
4,2023-10-16 18:21:28.482,1.15708,1.15745,864130.0,864090.0
...,...,...,...,...,...
61,2023-10-16 18:24:12.464,1.15695,1.15732,950642.0,864180.0
62,2023-10-16 18:24:14.683,1.15700,1.15737,864180.0,1728300.0
63,2023-10-16 18:24:15.649,1.15701,1.15738,864180.0,1728280.0
64,2023-10-16 18:24:16.753,1.15701,1.15738,864180.0,1728280.0


In [67]:
round_sig

<function __main__.round_sig(x, sig=10)>